In [1]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
client = MongoClient('rk5', 37373)
import matplotlib
matplotlib.use('pgf')
pgf_with_custom_preamble = {
    "font.family": "serif", # use serif/main font for text elements
    "text.usetex": True,    # use inline math for ticks
    'font.size' : 8,
    "pgf.rcfonts": False,   # don't setup fonts from rc parameters
    'figure.autolayout': True,
    'axes.labelsize': 8,'axes.titlesize':8, 
     'legend.fontsize': 8, 'xtick.labelsize': 8, 
     'ytick.labelsize': 8,
    "pgf.preamble": [
         "\\usepackage{units}",         # load additional packages
         "\\usepackage{metalogo}",
         "\\usepackage{unicode-math}",  # unicode math setup
         #r"\setmathfont{xits-math.lfg}",
         r"\setmainfont{Times New Roman}", # serif font via preamble
         r"\setmainfont{Times New Roman}", # serif font via preamble

         ]
}
matplotlib.rcParams.update(pgf_with_custom_preamble)

import matplotlib.pyplot as plt


In [2]:
import torch
import incense
from incense import ExperimentLoader
from incense import utils
import librosa
loader= ExperimentLoader(
    mongo_uri='rk5:37373',    
    db_name='d19t1fa'
)
    
from shutil import copyfile
import os
import pandas as pd
import sys
sys.path.append('/home/khaled/kofta')
import torch
import os
pwd=!pwd
import matplotlib.pyplot as plt
from matplotlib.colors import NoNorm
import numpy as np
scalfac=4
from pymongo import MongoClient
from sacred import Experiment, SETTINGS
from sacred.dependencies import get_sources_from_local_dir
from torch.utils.data import RandomSampler, BatchSampler

import shared_globals
import  helpers.utils as utils
from helpers.utils import load_model, get_criterion, apply_linefeeds
from importlib import reload


SETTINGS.HOST_INFO.INCLUDE_GPU_INFO = False  # faster startup
client = MongoClient('rk5', 37373)
import pandas as pd
db=client.d19t2
dbruns=db['runs']
import logging
import jsonpickle.unpickler
Unpickler = jsonpickle.unpickler.Unpickler()
savefile_patern="cached/dense{}.npy"

from matplotlib.transforms import Affine2D
import mpl_toolkits.axisartist.floating_axes as floating_axes
import matplotlib.pyplot as plt

def avg_model(eid,print_conf=False,return_model=False,bestm="",lastm="",only_one=False,get_loader=False,submode=False):
    _eid=str(eid)
    e1=loader.find_by_id(eid)
    config=dict(e1.config)
    import  datasets.dcase19.t1.task1 as t1
    reload(t1)
    dataset=t1.dataset

    from strategies.default import Trainer
    config=dict(config)
    
    ex = Experiment("predicting",
                ingredients=[dataset],interactive=True
                ) 
    trainer=None
    #outdir="/home/khaled/"+config['out_dir']+"/models/swa_model_{}.pth"
    outdir="/home/khaled/"+config['out_dir']+"/models/last_model_{}.pth"
    
    print("oooooo:",outdir)
    print("epochs",config['epochs'])
    
    config['out_dir']=pwd[0]+"/efic_run"
    
    #datasets.testing.batch_config.batch_size=1
    config['datasets']['testing']['batch_config']['batch_size']=1
    config['load_model']=""
    if submode:
        print("\n\n\nsub mod\n\n\n")
        config['datasets']['sub']=dict(config['datasets']['leaderboard'])
        config['datasets']['sub']['dataset']="audiodataset.get_sub_dataset"
        print( config['datasets']['sub'])
    #print(config)
    dataset.add_config(config['audiodataset'])
    ex.add_config(config)
    @ex.command
    def get_trainer(_run, _config, _log, _rnd, _seed, epochs):
        shared_globals.logger = logging.getLogger('')
        _log.info("output dir: " + _config['out_dir'])
        t = Trainer(_config, _run, _rnd, _seed, ex)
        nonlocal trainer
        trainer=t
    cmd="get_trainer"
    if print_conf:
        cmd="print_config"
    r=ex.run(cmd)

    if print_conf:
        return 0,0
    m=trainer.bare_model
    for param in m.parameters():
        param.requires_grad = False
    totalsids=None
    total_pres=None
    counter=0
    statrt=300
#     if not os.path.isfile(outdir.format(statrt)):
#         statrt=249
#     if not os.path.isfile(outdir.format(statrt)):
#         statrt=329
    output_formt="preds_permodel/single/m{}_ep{}.pth"
    if submode:
        output_formt="preds_permodel/single/sub_m{}_ep{}.pth"

    if submode:
        train_loader=trainer.data_loaders["sub"]
    else:
        train_loader=trainer.data_loaders["leaderboard"]
    if get_loader:
        return train_loader
    for snapi in range(statrt,config['epochs'],5):
        if e1.metrics['training.accuracy'][snapi]<1:
            print("skipping epoch ",snapi)
            continue
        
        load_model_path=outdir.format(snapi)
        checkpoint = torch.load(os.path.expanduser(load_model_path))
        m.load_state_dict(checkpoint['state_dict'])
        m.eval()
        if submode:
            train_loader=trainer.data_loaders["sub"]
        else:
            train_loader=trainer.data_loaders["leaderboard"]
        if get_loader:
            return train_loader
        #train_loader=trainer.data_loaders["training"]
        #print("WARNING TRAINING SET")

        counter=0
        pred=[]
        sids=[]
        if not os.path.isfile(output_formt.format(_eid,snapi)):
            with torch.no_grad():
                for step, (data, sid, _) in enumerate(train_loader):
                    if step%200==0:
                        print("\r id: ",eid," snap ",snapi," step ",step,end="")
                    data = data.cuda()
                    #m.zero_grad()

                    outputs = m(data)
                    pred.append(outputs)
                    sids.append(sid[0])

            if totalsids is None:
                totalsids=sids
            assert totalsids==sids
            p=torch.cat(pred,dim=0)
            torch.save(p,output_formt.format(_eid,snapi))
        else:
            p=torch.load(output_formt.format(_eid,snapi))
        #total_pres.append(p)
        counter+=1
        if total_pres is None:
            total_pres=p
        else:
            total_pres+=p
        
        if only_one:
            break
    #np.save(savefile_patern.format(idf ),accum)    

    return total_pres, totalsids,counter

def figure_show(img,txt="effictive"):
    plt.figure(figsize=(14,10))
    plt.imshow(img/img.max(), cmap=plt.get_cmap('gray'),vmin=0,vmax=1)
    plt.colorbar()
    plt.xlabel(txt)
    plt.show()
def get_one(_id,dbruns=dbruns,recaluclate=False,sanity_check=True,vis_method=2,print_conf=0):
    #e=dbruns.find_one({'_id': _id})
    e=loader.find_by_id(_id)
    #e.metrics['training.accuracy'][140]
    i=str(_id)
    if not e:
        raise FileNotFoundError
    #e=Unpickler.restore(e)
    ex=e
    e=ex['config']
    #loader,model=
    return eff_rf("~"+e['info']['best_model_path'],dict(e['config']),i,print_conf=print_conf,return_model=1)

#avg_model(363)

In [3]:

e1=loader.find_by_id(21)

e1.status

'COMPLETED'

In [4]:
extra_to_max={
-7: 23,
-6: 31,
-5: 39,
-4: 55,
-3: 71,
-2: 87,
-1: 103,
0: 135,
1: 167,
2: 199,
3: 231,
4: 263,
5: 295,
6: 327,
7: 359,
8: 391,
9: 423,
10: 455,
11: 487,
12: 519,
13: 551,
14: 583}
for i,k in enumerate(extra_to_max.keys()):
    print(i,k)

0 -7
1 -6
2 -5
3 -4
4 -3
5 -2
6 -1
7 0
8 1
9 2
10 3
11 4
12 5
13 6
14 7
15 8
16 9
17 10
18 11
19 12
20 13
21 14


In [5]:
for i,k in enumerate(extra_to_max.keys()):
    print(i,"& $",extra_to_max[k]," \\times ",extra_to_max[k]," $")
    if i%2==1:
        print("\\\\ \hline")
    else:
        print("&")

0 & $ 23  \times  23  $
&
1 & $ 31  \times  31  $
\\ \hline
2 & $ 39  \times  39  $
&
3 & $ 55  \times  55  $
\\ \hline
4 & $ 71  \times  71  $
&
5 & $ 87  \times  87  $
\\ \hline
6 & $ 103  \times  103  $
&
7 & $ 135  \times  135  $
\\ \hline
8 & $ 167  \times  167  $
&
9 & $ 199  \times  199  $
\\ \hline
10 & $ 231  \times  231  $
&
11 & $ 263  \times  263  $
\\ \hline
12 & $ 295  \times  295  $
&
13 & $ 327  \times  327  $
\\ \hline
14 & $ 359  \times  359  $
&
15 & $ 391  \times  391  $
\\ \hline
16 & $ 423  \times  423  $
&
17 & $ 455  \times  455  $
\\ \hline
18 & $ 487  \times  487  $
&
19 & $ 519  \times  519  $
\\ \hline
20 & $ 551  \times  551  $
&
21 & $ 583  \times  583  $
\\ \hline


In [6]:
#saml={"config":[],"use_mixup":[],"arch":[],"ta":[],"tl":[]}
confs_arr=[]
for e1 in loader.find_all():
    if e1.status=='COMPLETED' and (e1.config.extra_kernal_rf>-7 and e1.config.extra_kernal_rf<5):
        print(e1.id,e1.config.extra_kernal_rf,e1.config.arch,e1.config.use_mixup,e1.status)
    else:
        continue
    for ta,tl in zip(e1.metrics['testing.accuracy'][345:].values,e1.metrics['testing.loss'][345:].values):
    #for ta,tl in zip([np.max(e1.metrics['testing.accuracy'][:].values)],[np.min(e1.metrics['testing.loss'][345:].values)]):
        
        ids=""
        if e1.config.use_mixup:
            ids+="zmixup_"
        else:
            ids+="a"
        if "locpad" in e1.config.arch:
            ids+="freq_aware"
        elif "cp_preactbnresnetshake"in e1.config.arch:
            ids+="shake-shake"
        elif "cp_resnet"==e1.config.arch:
            ids+="resnet"
        elif "preact"in e1.config.arch:
             ids+="preact"
        ids+="_"
        
        #saml["use_mixup"].append(e1.config.use_mixup)
        #saml["arch"].append(e1.config.arch)
        confs_arr.append({""+ids+"acc": ta,""+ids+"loss": tl,"aconfig":extra_to_max[e1.config.extra_kernal_rf]})

keys_gather=list(set.union(*map(set,[[x for x in a.keys()] for a in confs_arr])))
keys_gather.sort()
print(keys_gather)
saml=dict([(k[1:],[]) for k in keys_gather])
for it in confs_arr:
    for k in keys_gather:
        saml[k[1:]].append(it.get(k))
keys_gather=[k[1:]for k in keys_gather]
print(keys_gather)

1 -4 cp_resnet_locpad False COMPLETED
2 -3 cp_resnet_locpad False COMPLETED
3 -2 cp_resnet_locpad False COMPLETED
4 -1 cp_resnet_locpad False COMPLETED
5 -4 cp_resnet_locpad False COMPLETED
6 -3 cp_resnet_locpad False COMPLETED
7 -1 cp_resnet_locpad False COMPLETED
8 -2 cp_resnet_locpad False COMPLETED
9 -4 cp_resnet False COMPLETED
10 -3 cp_resnet False COMPLETED
11 -2 cp_resnet False COMPLETED
12 2 cp_resnet_locpad False COMPLETED
15 0 cp_resnet_locpad False COMPLETED
17 1 cp_resnet_locpad False COMPLETED
18 0 cp_resnet False COMPLETED
19 1 cp_resnet False COMPLETED
20 2 cp_resnet False COMPLETED
21 -1 cp_resnet_locpad True COMPLETED
22 -1 cp_resnet True COMPLETED
23 0 cp_resnet_locpad True COMPLETED
24 1 cp_resnet True COMPLETED
25 -1 cp_resnet_locpad True COMPLETED
26 -1 cp_resnet True COMPLETED
27 -4 cp_resnet False COMPLETED
28 -2 cp_resnet False COMPLETED
29 -3 cp_resnet False COMPLETED
30 -2 cp_resnet True COMPLETED
31 0 cp_resnet False COMPLETED
32 1 cp_resnet False COMPLETED


In [7]:
import pandas as pd

In [8]:
df=pd.DataFrame(saml)
df.describe()

config  freq_aware_acc  freq_aware_loss  preact_acc  preact_loss  \
count  820.000000      145.000000       145.000000  110.000000   110.000000   
mean   124.756098        0.788738         0.799398    0.792469     0.769376   
std     80.061265        0.019628         0.104748    0.019122     0.079747   
min     31.000000        0.741458         0.634261    0.734050     0.641668   
25%     55.000000        0.772760         0.703928    0.788710     0.704974   
50%    103.000000        0.793787         0.779704    0.799881     0.761498   
75%    199.000000        0.804779         0.908922    0.804301     0.828078   
max    263.000000        0.822939         0.993042    0.814098     0.948187   

       resnet_acc  resnet_loss  shake-shake_acc  shake-shake_loss  \
count  120.000000   120.000000        55.000000         55.000000   
mean     0.792061     0.770563         0.788891          0.772895   
std      0.019707     0.089794         0.009392          0.075952   
min      0.732616     0.630169         0.767025          0.660894   
25%      0.787336     0.695941         0.783751          0.715602   
50%      0.797969     0.765923         0.791398          0.744761   
75%      0.805795     0.813759         0.795579          0.823682   
max      0.815054     0.983256         0.804062          0.947732   

       mixup_freq_aware_acc  mixup_freq_aware_loss  mixup_preact_acc  \
count            140.000000             140.000000        110.000000   
mean               0.807194               0.581980          0.811183   
std                0.011745               0.044655          0.010282   
min                0.786380               0.503275          0.789964   
25%                0.798268               0.542032          0.802867   
50%                0.803584               0.582999          0.811111   
75%                0.817742               0.618230          0.819713   
max                0.832258               0.674618          0.831302   

       mixup_preact_loss  mixup_resnet_acc  mixup_resnet_loss  \
count         110.000000        100.000000         100.000000   
mean            0.573274          0.812631           0.566406   
std             0.047940          0.012472           0.051512   
min             0.493651          0.785663           0.497915   
25%             0.532193          0.805317           0.518200   
50%             0.573957          0.814934           0.556628   
75%             0.613624          0.822521           0.608083   
max             0.679635          0.832497           0.673320   

       mixup_shake-shake_acc  mixup_shake-shake_loss  
count              40.000000               40.000000  
mean                0.796434                0.619739  
std                 0.006793                0.042061  
min                 0.784229                0.551463  
25%                 0.791697                0.584142  
50%                 0.795938                0.612855  
75%                 0.801553                0.663111  
max                 0.810275                0.681489

In [9]:

gall=df[[k for k in keys_gather if "loss" in k]+["config"]].groupby(["config"])
#g=gall[gall["use_mixup"]==True]
g=gall
display(g.describe())
means = g.mean()
errors = g.std()


p = means.plot(figsize=(16,12), yerr = errors,legend=True,alpha=0.8)
#reset color cycle so that the marker colors match
#ax.set_prop_cycle(None)
#plot the markers
#means.plot(figsize=(4,3.2), style=['^-', 'o-', 'x-'], markersize=2, ax = ax)
#p.set_xticks(range(-7,15));
p.set_ylabel("Testing loss")
p.set_xlabel("Filters changed to 1x1")




#p.set_ylim([0.6,1.0])
p

freq_aware_loss                                                    \
                 count      mean       std       min       25%       50%   
config                                                                     
31                20.0  0.770614  0.030166  0.729745  0.737413  0.777868   
39                20.0  0.770898  0.025666  0.717427  0.751015  0.768750   
55                10.0  0.662988  0.018167  0.638894  0.650991  0.660299   
71                10.0  0.684041  0.038189  0.634261  0.648982  0.691046   
87                10.0  0.681233  0.025952  0.657783  0.667837  0.675577   
103               10.0  0.688836  0.013993  0.673477  0.679405  0.681320   
135                5.0  0.702079  0.015315  0.682151  0.699399  0.699908   
167               10.0  0.818964  0.043424  0.766456  0.780439  0.817475   
199               10.0  0.886705  0.046051  0.834779  0.845474  0.882051   
231               20.0  0.929356  0.019616  0.906238  0.915370  0.924442   
263               20.0  0.937862  0.032029  0.890468  0.912199  0.928099   

                           preact_loss            ... mixup_resnet_loss  \
             75%       max       count      mean  ...               75%   
config                                            ...                     
31      0.790988  0.822048        10.0  0.789970  ...          0.617033   
39      0.790711  0.819730        10.0  0.767481  ...          0.584980   
55      0.676483  0.694795        10.0  0.665681  ...          0.527776   
71      0.711735  0.736706        10.0  0.698109  ...          0.515090   
87      0.683741  0.748395        10.0  0.685257  ...          0.527011   
103     0.699547  0.713777        10.0  0.713787  ...          0.523185   
135     0.703928  0.725010        10.0  0.734750  ...          0.531115   
167     0.859172  0.873979        10.0  0.794051  ...          0.563425   
199     0.919660  0.971466        10.0  0.820249  ...          0.569751   
231     0.937411  0.980412        10.0  0.889962  ...          0.617367   
263     0.966888  0.993042        10.0  0.903839  ...          0.671578   

                 mixup_shake-shake_loss                                \
             max                  count      mean       std       min   
config                                                                  
31      0.622937                    5.0  0.611357  0.005236  0.602194   
39      0.592368                    5.0  0.578368  0.005430  0.571583   
55      0.534370                    0.0       NaN       NaN       NaN   
71      0.516394                    5.0  0.558262  0.006369  0.551463   
87      0.527414                    0.0       NaN       NaN       NaN   
103     0.541266                    5.0  0.593425  0.008312  0.579059   
135     0.531608                    5.0  0.612278  0.002690  0.608457   
167     0.569016                    5.0  0.670061  0.011655  0.651378   
199     0.585382                    5.0  0.673076  0.007733  0.661157   
231     0.628668                    5.0  0.661082  0.011594  0.644707   
263     0.673320                    0.0       NaN       NaN       NaN   

                                                
             25%       50%       75%       max  
config                                          
31      0.612562  0.612691  0.614148  0.615187  
39      0.574304  0.579276  0.581734  0.584945  
55           NaN       NaN       NaN       NaN  
71      0.552383  0.558752  0.562489  0.566223  
87           NaN       NaN       NaN       NaN  
103     0.593483  0.597603  0.597725  0.599256  
135     0.610664  0.613019  0.614318  0.614929  
167     0.668975  0.670267  0.678931  0.680756  
199     0.670392  0.676140  0.676204  0.681489  
231     0.657377  0.658935  0.670748  0.673644  
263          NaN       NaN       NaN       NaN  

[11 rows x 64 columns]

/home/khaled/miniconda3/envs/build_kofta/lib/python3.6/site-packages/numpy/core/numeric.py:492: UserWarning: Warning: converting a masked element to nan.
  return array(a, dtype, copy=False, order=order)


In [10]:

gall=df[[k for k in keys_gather if "acc" in k]+["config"]].groupby(["config"])
#g=gall[gall["use_mixup"]==True]
g=gall
display(g.describe())
means = g.mean()
errors = g.std()


p = means.plot(figsize=(16,12), yerr = errors,legend=True,alpha=0.8)
#reset color cycle so that the marker colors match
#ax.set_prop_cycle(None)
#plot the markers
#means.plot(figsize=(4,3.2), style=['^-', 'o-', 'x-'], markersize=2, ax = ax)
#p.set_xticks(range(-7,15));
p.set_ylabel("Testing loss")
p.set_xlabel("Filters changed to 1x1")




#p.set_ylim([0.6,1.0])
p

freq_aware_acc                                                    \
                count      mean       std       min       25%       50%   
config                                                                    
31               20.0  0.756726  0.006381  0.741458  0.753226  0.757587   
39               20.0  0.765352  0.008216  0.749582  0.760514  0.766308   
55               10.0  0.803297  0.003902  0.797133  0.800418  0.803465   
71               10.0  0.801075  0.008638  0.791398  0.794026  0.798327   
87               10.0  0.810705  0.009786  0.789964  0.807766  0.813262   
103              10.0  0.807407  0.003398  0.802628  0.804480  0.807288   
135               5.0  0.807885  0.003237  0.803106  0.806213  0.809080   
167              10.0  0.802198  0.011125  0.782079  0.795818  0.804421   
199              10.0  0.789916  0.009681  0.769176  0.785305  0.790800   
231              20.0  0.791505  0.004963  0.776583  0.788292  0.792234   
263              20.0  0.795496  0.007493  0.784468  0.791039  0.795221   

                           preact_acc            ... mixup_resnet_acc  \
             75%       max      count      mean  ...              75%   
config                                           ...                    
31      0.762067  0.765114       10.0  0.749104  ...         0.799821   
39      0.772401  0.778256       10.0  0.762031  ...         0.813859   
55      0.806511  0.808841       10.0  0.802437  ...         0.827121   
71      0.807527  0.814098       10.0  0.799594  ...         0.827897   
87      0.817145  0.822939       10.0  0.805018  ...         0.826523   
103     0.810394  0.811947       10.0  0.803345  ...         0.821744   
135     0.809797  0.811231       10.0  0.807503  ...         0.820789   
167     0.811111  0.815293       10.0  0.802413  ...         0.814576   
199     0.796774  0.802151       10.0  0.803274  ...         0.818638   
231     0.794026  0.800717       10.0  0.792378  ...         0.807766   
263     0.799343  0.809080       10.0  0.790060  ...         0.790621   

                 mixup_shake-shake_acc                                \
             max                 count      mean       std       min   
config                                                                 
31      0.802151                   5.0  0.785806  0.001989  0.784229   
39      0.816487                   5.0  0.799092  0.003888  0.792593   
55      0.830346                   0.0       NaN       NaN       NaN   
71      0.832497                   5.0  0.805591  0.003173  0.801912   
87      0.828196                   0.0       NaN       NaN       NaN   
103     0.822222                   5.0  0.796655  0.002607  0.793070   
135     0.821266                   5.0  0.796559  0.002080  0.795221   
167     0.815293                   5.0  0.789056  0.001571  0.786858   
199     0.821266                   5.0  0.794886  0.002532  0.791876   
231     0.809080                   5.0  0.803823  0.002557  0.801434   
263     0.791637                   0.0       NaN       NaN       NaN   

                                                
             25%       50%       75%       max  
config                                          
31      0.784707  0.785424  0.785424  0.789247  
39      0.798566  0.800717  0.801195  0.802389  
55           NaN       NaN       NaN       NaN  
71      0.803584  0.805735  0.806452  0.810275  
87           NaN       NaN       NaN       NaN  
103     0.795938  0.796416  0.797611  0.800239  
135     0.795460  0.795938  0.795938  0.800239  
167     0.788769  0.788769  0.789725  0.791159  
199     0.793548  0.794504  0.795938  0.798566  
231     0.801912  0.802628  0.806213  0.806930  
263          NaN       NaN       NaN       NaN  

[11 rows x 64 columns]

In [11]:
gall

# no mixy mixy

In [12]:
#saml={"config":[],"use_mixup":[],"arch":[],"ta":[],"tl":[]}
confs_arr=[]
for e1 in loader.find_all():
    if e1.status=='COMPLETED' and (e1.config.extra_kernal_rf>-7 and e1.config.extra_kernal_rf<5):
        #print(e1.id,e1.config.extra_kernal_rf,e1.config.arch,e1.config.use_mixup,e1.status)
        pass
    else:
        continue
    if e1.config.use_mixup:
        continue
    for ta,tl in zip(e1.metrics['testing.accuracy'][345:].values,e1.metrics['testing.loss'][345:].values):
    #for ta,tl in zip([np.max(e1.metrics['testing.accuracy'][:].values)],[np.min(e1.metrics['testing.loss'][345:].values)]):
        
        ids=""
        if e1.config.use_mixup:
            ids+="zmixup_"
        else:
            ids+="a"
        if "locpad" in e1.config.arch:
            ids+="FAResNet"
        elif "cp_preactbnresnetshake"in e1.config.arch:
            ids+="Shake-Shake"
        elif "cp_resnet"==e1.config.arch:
            ids+="ResNet"
        elif "preact"in e1.config.arch:
             ids+="PreActResNet"
        #ids+="_"
        
        #saml["use_mixup"].append(e1.config.use_mixup)
        #saml["arch"].append(e1.config.arch)
        confs_arr.append({""+ids+"": tl,"aconfig":extra_to_max[e1.config.extra_kernal_rf]})

keys_gather=list(set.union(*map(set,[[x for x in a.keys()] for a in confs_arr])))
keys_gather.sort()
print(keys_gather)
saml=dict([(k[1:],[]) for k in keys_gather])
for it in confs_arr:
    for k in keys_gather:
        saml[k[1:]].append(it.get(k))
keys_gather=[k[1:]for k in keys_gather]
print(keys_gather)


#gall=df[[k for k in keys_gather if "loss" in k]+["config"]].groupby(["config"])
#g=gall[gall["use_mixup"]==True]
df=pd.DataFrame(saml)

g=df.groupby(["config"])
display(g.describe())
means = g.mean()
errors = g.std()


p = means.plot(figsize=(16,12), yerr = errors,legend=True,alpha=0.8)
#reset color cycle so that the marker colors match
#ax.set_prop_cycle(None)
#plot the markers
#means.plot(figsize=(4,3.2), style=['^-', 'o-', 'x-'], markersize=2, ax = ax)
#p.set_xticks(range(-7,15));
p.set_ylabel("Testing loss")
p.set_xlabel("Max Receptive Field")




#p.set_ylim([0.6,1.0])
p

['aFAResNet', 'aPreActResNet', 'aResNet', 'aShake-Shake', 'aconfig']
['FAResNet', 'PreActResNet', 'ResNet', 'Shake-Shake', 'config']


FAResNet                                                              \
          count      mean       std       min       25%       50%       75%   
config                                                                        
31         20.0  0.770614  0.030166  0.729745  0.737413  0.777868  0.790988   
39         20.0  0.770898  0.025666  0.717427  0.751015  0.768750  0.790711   
55         10.0  0.662988  0.018167  0.638894  0.650991  0.660299  0.676483   
71         10.0  0.684041  0.038189  0.634261  0.648982  0.691046  0.711735   
87         10.0  0.681233  0.025952  0.657783  0.667837  0.675577  0.683741   
103        10.0  0.688836  0.013993  0.673477  0.679405  0.681320  0.699547   
135         5.0  0.702079  0.015315  0.682151  0.699399  0.699908  0.703928   
167        10.0  0.818964  0.043424  0.766456  0.780439  0.817475  0.859172   
199        10.0  0.886705  0.046051  0.834779  0.845474  0.882051  0.919660   
231        20.0  0.929356  0.019616  0.906238  0.915370  0.924442  0.937411   
263        20.0  0.937862  0.032029  0.890468  0.912199  0.928099  0.966888   

                 PreActResNet            ...    ResNet           Shake-Shake  \
             max        count      mean  ...       75%       max       count   
config                                   ...                                   
31      0.822048         10.0  0.789970  ...  0.789636  0.830599         5.0   
39      0.819730         10.0  0.767481  ...  0.786325  0.804612         5.0   
55      0.694795         10.0  0.665681  ...  0.684765  0.965775         5.0   
71      0.736706         10.0  0.698109  ...  0.739541  0.748870         5.0   
87      0.748395         10.0  0.685257  ...  0.680714  0.704214         5.0   
103     0.713777         10.0  0.713787  ...  0.710625  0.717146        10.0   
135     0.725010         10.0  0.734750  ...  0.792878  0.812694         5.0   
167     0.873979         10.0  0.794051  ...  0.800908  0.832512         5.0   
199     0.971466         10.0  0.820249  ...  0.865542  0.912707         5.0   
231     0.980412         10.0  0.889962  ...  0.910561  0.923726         5.0   
263     0.993042         10.0  0.903839  ...  0.950874  0.983256         0.0   

                                                                              
            mean       std       min       25%       50%       75%       max  
config                                                                        
31      0.708460  0.014066  0.692923  0.703383  0.707273  0.707418  0.731301  
39      0.719508  0.008548  0.713692  0.714894  0.716304  0.718141  0.734509  
55      0.668623  0.005068  0.660894  0.666812  0.669785  0.671529  0.674094  
71      0.715803  0.003877  0.711976  0.712951  0.714954  0.717533  0.721603  
87      0.720765  0.003854  0.716250  0.717679  0.721230  0.722949  0.725716  
103     0.772047  0.025984  0.737237  0.748785  0.774709  0.793751  0.806021  
135     0.821399  0.009241  0.805429  0.822843  0.824522  0.824894  0.829305  
167     0.807010  0.012247  0.794809  0.799633  0.801559  0.814341  0.824707  
199     0.871154  0.015312  0.846751  0.868664  0.873771  0.879136  0.887449  
231     0.925030  0.018688  0.902550  0.910103  0.927626  0.937140  0.947732  
263          NaN       NaN       NaN       NaN       NaN       NaN       NaN  

[11 rows x 32 columns]

In [13]:
#saml={"config":[],"use_mixup":[],"arch":[],"ta":[],"tl":[]}
confs_arr=[]
for e1 in loader.find_all():
    if e1.status=='COMPLETED' and (e1.config.extra_kernal_rf>-7 and e1.config.extra_kernal_rf<5):
        #print(e1.id,e1.config.extra_kernal_rf,e1.config.arch,e1.config.use_mixup,e1.status)
        pass
    else:
        continue
    if  e1.config.use_mixup:
        continue
    for ta,tl in zip(e1.metrics['testing.accuracy'][345:].values,e1.metrics['testing.loss'][345:].values):
    #for ta,tl in zip([np.max(e1.metrics['testing.accuracy'][:].values)],[np.min(e1.metrics['testing.loss'][345:].values)]):
        
        ids=""
        if e1.config.use_mixup:
            ids+="a"
        else:
            ids+="a"
        if "locpad" in e1.config.arch:
            ids+="FAResNet"
        elif "cp_preactbnresnetshake"in e1.config.arch:
            ids+="Shake-Shake"
        elif "cp_resnet"==e1.config.arch:
            ids+="ResNet"
        elif "preact"in e1.config.arch:
             ids+="PreActResNet"
        #ids+="_"
        
        #saml["use_mixup"].append(e1.config.use_mixup)
        #saml["arch"].append(e1.config.arch)
        confs_arr.append({""+ids+"": tl,"aconfig":extra_to_max[e1.config.extra_kernal_rf]})

keys_gather=list(set.union(*map(set,[[x for x in a.keys()] for a in confs_arr])))
keys_gather.sort()
print(keys_gather)
saml=dict([(k[1:],[]) for k in keys_gather])
for it in confs_arr:
    for k in keys_gather:
        saml[k[1:]].append(it.get(k))
keys_gather=[k[1:]for k in keys_gather]
print(keys_gather)


#gall=df[[k for k in keys_gather if "loss" in k]+["config"]].groupby(["config"])
#g=gall[gall["use_mixup"]==True]
df=pd.DataFrame(saml)

g=df.groupby(["config"])
display(g.describe())
means = g.mean()
errors = g.std()


p = means.plot(figsize=(16,12), yerr = errors,legend=True,alpha=0.8)
#reset color cycle so that the marker colors match
#ax.set_prop_cycle(None)
#plot the markers
#means.plot(figsize=(4,3.2), style=['^-', 'o-', 'x-'], markersize=2, ax = ax)
#p.set_xticks(range(-7,15));
p.set_ylabel("Testing loss")
p.set_xlabel("Max Receptive Field")




#p.set_ylim([0.6,1.0])
p


['aFAResNet', 'aPreActResNet', 'aResNet', 'aShake-Shake', 'aconfig']
['FAResNet', 'PreActResNet', 'ResNet', 'Shake-Shake', 'config']


FAResNet                                                              \
          count      mean       std       min       25%       50%       75%   
config                                                                        
31         20.0  0.770614  0.030166  0.729745  0.737413  0.777868  0.790988   
39         20.0  0.770898  0.025666  0.717427  0.751015  0.768750  0.790711   
55         10.0  0.662988  0.018167  0.638894  0.650991  0.660299  0.676483   
71         10.0  0.684041  0.038189  0.634261  0.648982  0.691046  0.711735   
87         10.0  0.681233  0.025952  0.657783  0.667837  0.675577  0.683741   
103        10.0  0.688836  0.013993  0.673477  0.679405  0.681320  0.699547   
135         5.0  0.702079  0.015315  0.682151  0.699399  0.699908  0.703928   
167        10.0  0.818964  0.043424  0.766456  0.780439  0.817475  0.859172   
199        10.0  0.886705  0.046051  0.834779  0.845474  0.882051  0.919660   
231        20.0  0.929356  0.019616  0.906238  0.915370  0.924442  0.937411   
263        20.0  0.937862  0.032029  0.890468  0.912199  0.928099  0.966888   

                 PreActResNet            ...    ResNet           Shake-Shake  \
             max        count      mean  ...       75%       max       count   
config                                   ...                                   
31      0.822048         10.0  0.789970  ...  0.789636  0.830599         5.0   
39      0.819730         10.0  0.767481  ...  0.786325  0.804612         5.0   
55      0.694795         10.0  0.665681  ...  0.684765  0.965775         5.0   
71      0.736706         10.0  0.698109  ...  0.739541  0.748870         5.0   
87      0.748395         10.0  0.685257  ...  0.680714  0.704214         5.0   
103     0.713777         10.0  0.713787  ...  0.710625  0.717146        10.0   
135     0.725010         10.0  0.734750  ...  0.792878  0.812694         5.0   
167     0.873979         10.0  0.794051  ...  0.800908  0.832512         5.0   
199     0.971466         10.0  0.820249  ...  0.865542  0.912707         5.0   
231     0.980412         10.0  0.889962  ...  0.910561  0.923726         5.0   
263     0.993042         10.0  0.903839  ...  0.950874  0.983256         0.0   

                                                                              
            mean       std       min       25%       50%       75%       max  
config                                                                        
31      0.708460  0.014066  0.692923  0.703383  0.707273  0.707418  0.731301  
39      0.719508  0.008548  0.713692  0.714894  0.716304  0.718141  0.734509  
55      0.668623  0.005068  0.660894  0.666812  0.669785  0.671529  0.674094  
71      0.715803  0.003877  0.711976  0.712951  0.714954  0.717533  0.721603  
87      0.720765  0.003854  0.716250  0.717679  0.721230  0.722949  0.725716  
103     0.772047  0.025984  0.737237  0.748785  0.774709  0.793751  0.806021  
135     0.821399  0.009241  0.805429  0.822843  0.824522  0.824894  0.829305  
167     0.807010  0.012247  0.794809  0.799633  0.801559  0.814341  0.824707  
199     0.871154  0.015312  0.846751  0.868664  0.873771  0.879136  0.887449  
231     0.925030  0.018688  0.902550  0.910103  0.927626  0.937140  0.947732  
263          NaN       NaN       NaN       NaN       NaN       NaN       NaN  

[11 rows x 32 columns]

In [14]:
#saml={"config":[],"use_mixup":[],"arch":[],"ta":[],"tl":[]}
confs_arr=[]
for e1 in loader.find_all():
    if e1.status=='COMPLETED' and (e1.config.extra_kernal_rf>-7 and e1.config.extra_kernal_rf<5):
        #print(e1.id,e1.config.extra_kernal_rf,e1.config.arch,e1.config.use_mixup,e1.status)
        pass
    else:
        continue
    if not e1.config.use_mixup:
        continue
    for ta,tl in zip(e1.metrics['testing.accuracy'][345:].values,e1.metrics['testing.loss'][345:].values):
    #for ta,tl in zip([np.max(e1.metrics['testing.accuracy'][:].values)],[np.min(e1.metrics['testing.loss'][345:].values)]):
        
        ids=""
        if e1.config.use_mixup:
            ids+="a"
        else:
            ids+="a"
        if "locpad" in e1.config.arch:
            ids+="FAResNet"
        elif "cp_preactbnresnetshake"in e1.config.arch:
            ids+="Shake-Shake"
        elif "cp_resnet"==e1.config.arch:
            ids+="ResNet"
        elif "preact"in e1.config.arch:
             ids+="PreActResNet"
        #ids+="_"
        
        #saml["use_mixup"].append(e1.config.use_mixup)
        #saml["arch"].append(e1.config.arch)
        confs_arr.append({""+ids+"": tl,"aconfig":extra_to_max[e1.config.extra_kernal_rf]})

keys_gather=list(set.union(*map(set,[[x for x in a.keys()] for a in confs_arr])))
keys_gather.sort()
print(keys_gather)
saml=dict([(k[1:],[]) for k in keys_gather])
for it in confs_arr:
    for k in keys_gather:
        saml[k[1:]].append(it.get(k))
keys_gather=[k[1:]for k in keys_gather]
print(keys_gather)


#gall=df[[k for k in keys_gather if "loss" in k]+["config"]].groupby(["config"])
#g=gall[gall["use_mixup"]==True]
df=pd.DataFrame(saml)

g=df.groupby(["config"])
display(g.describe())
means = g.mean()
errors = g.std()


p = means.plot(figsize=(16,12), yerr = errors,legend=True,alpha=0.8)
#reset color cycle so that the marker colors match
#ax.set_prop_cycle(None)
#plot the markers
#means.plot(figsize=(4,3.2), style=['^-', 'o-', 'x-'], markersize=2, ax = ax)
#p.set_xticks(range(-7,15));
p.set_ylabel("Testing loss")
p.set_xlabel("Max Receptive Field")




#p.set_ylim([0.6,1.0])
p

#saml={"config":[],"use_mixup":[],"arch":[],"ta":[],"tl":[]}
confs_arr=[]
for e1 in loader.find_all():
    if e1.status=='COMPLETED' and (e1.config.extra_kernal_rf>-7 and e1.config.extra_kernal_rf<5):
        #print(e1.id,e1.config.extra_kernal_rf,e1.config.arch,e1.config.use_mixup,e1.status)
        pass
    else:
        continue
    if not e1.config.use_mixup:
        continue
    for ta,tl in zip(e1.metrics['testing.accuracy'][345:].values,e1.metrics['testing.loss'][345:].values):
    #for ta,tl in zip([np.max(e1.metrics['testing.accuracy'][:].values)],[np.min(e1.metrics['testing.loss'][345:].values)]):
        
        ids=""
        if e1.config.use_mixup:
            ids+="zmixup_"
        else:
            ids+="a"
        if "locpad" in e1.config.arch:
            ids+="FAResNet"
        elif "cp_preactbnresnetshake"in e1.config.arch:
            ids+="Shake-Shake"
        elif "cp_resnet"==e1.config.arch:
            ids+="ResNet"
        elif "preact"in e1.config.arch:
             ids+="PreActResNet"
        #ids+="_"
        
        #saml["use_mixup"].append(e1.config.use_mixup)
        #saml["arch"].append(e1.config.arch)
        confs_arr.append({""+ids+"": ta,"aconfig":extra_to_max[e1.config.extra_kernal_rf]})

keys_gather=list(set.union(*map(set,[[x for x in a.keys()] for a in confs_arr])))
keys_gather.sort()
print(keys_gather)
saml=dict([(k[1:],[]) for k in keys_gather])
for it in confs_arr:
    for k in keys_gather:
        saml[k[1:]].append(it.get(k))
keys_gather=[k[1:]for k in keys_gather]
print(keys_gather)


#gall=df[[k for k in keys_gather if "loss" in k]+["config"]].groupby(["config"])
#g=gall[gall["use_mixup"]==True]
df=pd.DataFrame(saml)

g=df.groupby(["config"])
display(g.describe())
means2 = g.mean()
errors2 = g.std()


p = means2.plot(figsize=(16,12), yerr = errors2,legend=True,alpha=0.8)
#reset color cycle so that the marker colors match
#ax.set_prop_cycle(None)
#plot the markers
#means.plot(figsize=(4,3.2), style=['^-', 'o-', 'x-'], markersize=2, ax = ax)
#p.set_xticks(range(-7,15));
p.set_ylabel("Testing loss")
p.set_xlabel("Max Receptive Field")




#p.set_ylim([0.6,1.0])
p

['aFAResNet', 'aPreActResNet', 'aResNet', 'aShake-Shake', 'aconfig']
['FAResNet', 'PreActResNet', 'ResNet', 'Shake-Shake', 'config']


FAResNet                                                              \
          count      mean       std       min       25%       50%       75%   
config                                                                        
31         20.0  0.601979  0.013125  0.565767  0.597167  0.603077  0.610696   
39         20.0  0.570166  0.012700  0.549569  0.561074  0.570046  0.575154   
55         10.0  0.543547  0.007723  0.530460  0.537159  0.544922  0.549814   
71         10.0  0.521545  0.007784  0.505660  0.517408  0.523216  0.525565   
87         10.0  0.523914  0.006849  0.512288  0.520073  0.526560  0.528182   
103        10.0  0.518714  0.012605  0.503275  0.506930  0.519446  0.529360   
135         5.0  0.544620  0.010640  0.526812  0.544733  0.546437  0.550770   
167         5.0  0.587364  0.009095  0.576564  0.582797  0.584131  0.594438   
199         5.0  0.581179  0.004091  0.576021  0.578305  0.581972  0.583180   
231        25.0  0.617609  0.009033  0.600796  0.610626  0.620048  0.624943   
263        20.0  0.647551  0.015249  0.629151  0.634115  0.642382  0.661716   

                 PreActResNet            ...    ResNet           Shake-Shake  \
             max        count      mean  ...       75%       max       count   
config                                   ...                                   
31      0.625240         10.0  0.618688  ...  0.617033  0.622937         5.0   
39      0.602093         10.0  0.585316  ...  0.584980  0.592368         5.0   
55      0.554140         10.0  0.543308  ...  0.527776  0.534370         0.0   
71      0.531663         10.0  0.523525  ...  0.515090  0.516394         5.0   
87      0.533613         10.0  0.513699  ...  0.527011  0.527414         0.0   
103     0.535324         10.0  0.508382  ...  0.523185  0.541266         5.0   
135     0.554346         10.0  0.551234  ...  0.531115  0.531608         5.0   
167     0.598891         10.0  0.576292  ...  0.563425  0.569016         5.0   
199     0.586418         10.0  0.607193  ...  0.569751  0.585382         5.0   
231     0.629515         10.0  0.627886  ...  0.617367  0.628668         5.0   
263     0.674618         10.0  0.650495  ...  0.671578  0.673320         0.0   

                                                                              
            mean       std       min       25%       50%       75%       max  
config                                                                        
31      0.611357  0.005236  0.602194  0.612562  0.612691  0.614148  0.615187  
39      0.578368  0.005430  0.571583  0.574304  0.579276  0.581734  0.584945  
55           NaN       NaN       NaN       NaN       NaN       NaN       NaN  
71      0.558262  0.006369  0.551463  0.552383  0.558752  0.562489  0.566223  
87           NaN       NaN       NaN       NaN       NaN       NaN       NaN  
103     0.593425  0.008312  0.579059  0.593483  0.597603  0.597725  0.599256  
135     0.612278  0.002690  0.608457  0.610664  0.613019  0.614318  0.614929  
167     0.670061  0.011655  0.651378  0.668975  0.670267  0.678931  0.680756  
199     0.673076  0.007733  0.661157  0.670392  0.676140  0.676204  0.681489  
231     0.661082  0.011594  0.644707  0.657377  0.658935  0.670748  0.673644  
263          NaN       NaN       NaN       NaN       NaN       NaN       NaN  

[11 rows x 32 columns]

['aconfig', 'zmixup_FAResNet', 'zmixup_PreActResNet', 'zmixup_ResNet', 'zmixup_Shake-Shake']
['config', 'mixup_FAResNet', 'mixup_PreActResNet', 'mixup_ResNet', 'mixup_Shake-Shake']


mixup_FAResNet                                                    \
                count      mean       std       min       25%       50%   
config                                                                    
31               20.0  0.794827  0.003778  0.787575  0.792712  0.794265   
39               20.0  0.803321  0.004784  0.795699  0.800418  0.802270   
55               10.0  0.818686  0.002836  0.812186  0.817682  0.819355   
71               10.0  0.826571  0.002672  0.823417  0.824313  0.826404   
87               10.0  0.820048  0.003384  0.816249  0.817563  0.819235   
103              10.0  0.822676  0.006094  0.814815  0.817264  0.822581   
135               5.0  0.823226  0.002928  0.819594  0.821744  0.823178   
167               5.0  0.807599  0.002514  0.804062  0.806452  0.807646   
199               5.0  0.811087  0.001358  0.809319  0.810992  0.810992   
231              25.0  0.801157  0.002989  0.796655  0.799283  0.800478   
263              20.0  0.796296  0.005027  0.786380  0.792473  0.797133   

                           mixup_PreActResNet            ... mixup_ResNet  \
             75%       max              count      mean  ...          75%   
config                                                   ...                
31      0.796177  0.805735               10.0  0.795986  ...     0.799821   
39      0.807826  0.810992               10.0  0.801004  ...     0.813859   
55      0.819773  0.822700               10.0  0.812593  ...     0.827121   
71      0.827957  0.832258               10.0  0.823632  ...     0.827897   
87      0.821983  0.826762               10.0  0.823728  ...     0.826523   
103     0.827360  0.831063               10.0  0.825496  ...     0.821744   
135     0.824134  0.827479               10.0  0.815771  ...     0.820789   
167     0.809319  0.810514               10.0  0.812282  ...     0.814576   
199     0.810992  0.813142               10.0  0.809008  ...     0.818638   
231     0.803106  0.809319               10.0  0.802628  ...     0.807766   
263     0.799582  0.804301               10.0  0.800884  ...     0.790621   

                 mixup_Shake-Shake                                          \
             max             count      mean       std       min       25%   
config                                                                       
31      0.802151               5.0  0.785806  0.001989  0.784229  0.784707   
39      0.816487               5.0  0.799092  0.003888  0.792593  0.798566   
55      0.830346               0.0       NaN       NaN       NaN       NaN   
71      0.832497               5.0  0.805591  0.003173  0.801912  0.803584   
87      0.828196               0.0       NaN       NaN       NaN       NaN   
103     0.822222               5.0  0.796655  0.002607  0.793070  0.795938   
135     0.821266               5.0  0.796559  0.002080  0.795221  0.795460   
167     0.815293               5.0  0.789056  0.001571  0.786858  0.788769   
199     0.821266               5.0  0.794886  0.002532  0.791876  0.793548   
231     0.809080               5.0  0.803823  0.002557  0.801434  0.801912   
263     0.791637               0.0       NaN       NaN       NaN       NaN   

                                      
             50%       75%       max  
config                                
31      0.785424  0.785424  0.789247  
39      0.800717  0.801195  0.802389  
55           NaN       NaN       NaN  
71      0.805735  0.806452  0.810275  
87           NaN       NaN       NaN  
103     0.796416  0.797611  0.800239  
135     0.795938  0.795938  0.800239  
167     0.788769  0.789725  0.791159  
199     0.794504  0.795938  0.798566  
231     0.802628  0.806213  0.806930  
263          NaN       NaN       NaN  

[11 rows x 32 columns]

In [18]:
fig, axs = plt.subplots(1, 2, figsize=(7.3, 3.2), sharey=False)
p = means2.plot(figsize=(7.3, 3.2), yerr = errors2,legend=True,alpha=0.8,ax=axs[0])
p.set_ylabel("Testing Accuracy")
p.set_xlabel("Max Receptive Field")


p = means.plot(figsize=(7.3, 3.2), yerr = errors,legend=False,alpha=0.8,ax=axs[1])
p.set_ylabel("Testing loss")
p.set_xlabel("Max Receptive Field")

plt.savefig("mixup.pdf")


/home/khaled/miniconda3/envs/build_kofta/lib/python3.6/site-packages/numpy/core/numeric.py:492: UserWarning: Warning: converting a masked element to nan.
  return array(a, dtype, copy=False, order=order)
